In [22]:
from typing import List
import whisper

model = whisper.load_model("tiny")

In [23]:
COHERE_API_KEY = "8rLIrm60Gf9mSmeIF2RHxv1TNcjXpHFDZ9XnqjD2"

In [24]:
def timestamps2summaries(video_url: str, timestamps: List[int], output_dir: str):
  transcribed = model.transcribe(video_url)
  print(transcribed)
  # transcribed["timestamps"]

In [4]:
video_url = "videos/cs61a_lec1.mkv"
# video_url = "videos/cs61b_lec1.webm"

result = model.transcribe(video_url)

/Users/lawrencechen/fun/autochapter/venv/lib/python3.10/site-packages/whisper/transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [26]:
import json

with open("out.json", "w") as f:
  f.write(json.dumps(result))

In [6]:
result["segments"]

[{'id': 0,
  'seek': 0,
  'start': 0.0,
  'end': 2.72,
  'text': " So today we're going to be doing a software engineering lecture.",
  'tokens': [50364,
   407,
   965,
   321,
   434,
   516,
   281,
   312,
   884,
   257,
   4722,
   7043,
   7991,
   13,
   50500,
   50500,
   639,
   307,
   257,
   8344,
   295,
   364,
   5120,
   13,
   50580,
   50580,
   407,
   286,
   478,
   516,
   281,
   976,
   291,
   512,
   646,
   4417,
   418,
   293,
   983,
   309,
   311,
   2737,
   13,
   50704,
   50704,
   407,
   1880,
   3637,
   300,
   294,
   16416,
   11,
   50916,
   50916,
   398,
   742,
   293,
   3071,
   11,
   28294,
   32,
   632,
   732,
   1708,
   257,
   1243,
   11,
   8138,
   11,
   51142,
   51142,
   10579,
   82,
   295,
   6191,
   8378,
   13,
   51240,
   51240,
   400,
   550,
   6984,
   390,
   1009,
   746,
   1646,
   13,
   51350,
   51350,
   823,
   2171,
   300,
   390,
   512,
   2093,
   16602,
   551,
   51498,
   51498,
   689,
   32

In [18]:
import cohere
co = cohere.Client(COHERE_API_KEY)

def generate_summary(passage: str):
  response = co.generate(
    model='large',
    prompt=f'Passage: Is Wordle getting tougher to solve? Players seem to be convinced that the game has gotten harder in recent weeks ever since The New York Times bought it from developer Josh Wardle in late January. The Times has come forward and shared that this likely isn’t the case. That said, the NYT did mess with the back end code a bit, removing some offensive and sexual language, as well as some obscure words There is a viral thread claiming that a confirmation bias was at play. One Twitter user went so far as to claim the game has gone to “the dusty section of the dictionary” to find its latest words.\n\nTLDR: Wordle has not gotten more difficult to solve.\n--\nPassage: ArtificialIvan, a seven-year-old, London-based payment and expense management software company, has raised $190 million in Series C funding led by ARG Global, with participation from D9 Capital Group and Boulder Capital. Earlier backers also joined the round, including Hilton Group, Roxanne Capital, Paved Roads Ventures, Brook Partners, and Plato Capital.\n\nTLDR: ArtificialIvan has raised $190 million in Series C funding.\n--\nPassage: {passage}\n\nTLDR:',
    max_tokens=50,
    temperature=0.8,
    k=0,
    p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop_sequences=["--"],
    return_likelihoods='NONE')
  # print('Prediction: {}'.format(response.generations[0].text))
  return response.generations[0].text

In [21]:
generate_summary(result["text"][0:2000])

' Rachael Bale is going to give you a set of lecture about software engineering.\n--'

In [ ]:
def get_segment(start: int, duration: int):
  return result["text"][start:start+duration]